Preparation of raw data such that a manageable size can be worked on with for target column preparation.

Flow
1. Load data (pdf - portfolio dataframe) that contains 'loan_account_no', 'report_date','number_of_installments','days_past_due','bucket_MA','bucket_RBI'.
2. Create reduced pdf (bc pdf is pdf is usually pretty large) and then save.
3. Check dates: confirm if 'repayment_start_date' > 'loan_application_date'
4. Check if number of installments is equal to (maturitydate - repayment_start_date + 1) [in year-month]
5. If 3-4 are confirmed, merge datasets
6. simplify columns to only include loan_account_no, repayment_start_date, report_date, bucket_MA, days_past_due]
7. create a column for nth month
8. save df with nth month and bucket, proceed with target column tagging (spearate notebook)

In [1]:
#import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
import datetime
import math

In [3]:
#load data
basis_df = pd.read_csv(r"C:\Users\emanlangit\Documents\ACSIN\01 ACSIN EDA\Data\BASE DATA FOR EDA.csv", low_memory=False)

In [4]:
basis_df = basis_df.dropna(subset = ['disbursal_amount'])

In [8]:
basis_df.shape

(87093, 168)

In [7]:
#load data
portfolio_df = pd.read_csv(r"C:\Users\emanlangit\Documents\ACSIN\01 ACSIN EDA\Data\portfolio.csv", low_memory=False)

In [9]:
portfolio_df.shape

(5614957, 30)

In [10]:
needed_cols = ['loan_account_no', 'report_date','number_of_installments','days_past_due','bucket_MA','bucket_RBI']
reduced_pdf = portfolio_df[needed_cols]
#reduced_pdf = pd.read_csv(r"C:\Users\emanlangit\Documents\ACSIN\Notebooks\reduced_data.csv")

In [13]:
reduced_pdf.head()

,loan_account_no,report_date,number_of_installments,days_past_due,bucket_MA,bucket_RBI
0,TW01020104000000005397,2021-01-01,12.0,0.0,D0,D0
1,TW01020102000000005590,2021-01-01,36.0,0.0,D0,D0
2,TW01010105000000005537,2021-01-01,36.0,0.0,D0,D0
3,TW01010104000000005899,2021-01-01,36.0,0.0,D0,D0
4,TW01010103000000005795,2021-01-01,24.0,0.0,D0,D0


In [ ]:
# #valid rows
basis_df = basis_df.dropna(subset = ['disbursal_amount'])
# valid_accounts = basis_df.loan_account_no.unique().tolist()
# reduced_pdf = reduced_pdf[reduced_pdf['loan_account_no'].isin(valid_accounts)]
# reduced_pdf.to_csv('valid_reduced_portfolio.csv')

In [ ]:
basis_df.columns.tolist()

In [ ]:
working_pdf = pd.read_csv(r"C:\Users\emanlangit\Documents\ACSIN\Notebooks\valid_reduced_portfolio.csv")
working_pdf.columns
working_pdf = working_pdf.drop(columns = ['Unnamed: 0.1', 'Unnamed: 0'])
working_pdf.head()
working_pdf.isnull().sum()
working_pdf.days_past_due.unique()
working_pdf.report_date
working_pdf.info()
len(working_pdf.loan_account_no.unique())
len(basis_df.loan_account_no.unique())

In [16]:
dates = basis_df[['repayment_start_date','loan_application_date']].apply(lambda x: pd.to_datetime(x, format='%Y-%m-%d', errors = 'coerce'))

In [17]:
dates['check 1'] = dates['repayment_start_date'] > dates['loan_application_date']

In [18]:
dates['check 1'].value_counts()

check 1
True    87093
Name: count, dtype: int64

In [20]:
#basis_df.maturitydate

In [21]:
#number_of_installments
check_df = basis_df[['loan_account_no','maturitydate','number_of_installments','repayment_start_date']]
check_df['maturitydate'] = check_df['maturitydate'].apply(lambda x: pd.to_datetime(x, format='%Y-%m-%d', errors = 'coerce'))
check_df['repayment_start_date'] = check_df['repayment_start_date'].apply(lambda x: pd.to_datetime(x, format='%Y-%m-%d', errors = 'coerce'))

C:\Users\emanlangit.AEONCREDIT\AppData\Local\Temp\ipykernel_3584\2574709305.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  check_df['maturitydate'] = check_df['maturitydate'].apply(lambda x: pd.to_datetime(x, format='%Y-%m-%d', errors = 'coerce'))
C:\Users\emanlangit.AEONCREDIT\AppData\Local\Temp\ipykernel_3584\2574709305.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  check_df['repayment_start_date'] = check_df['repayment_start_date'].apply(lambda x: pd.to_datetime(x, format='%Y-%m-%d', errors = '

In [22]:
check_df['diff'] = check_df['maturitydate'].dt.to_period('M') - check_df['repayment_start_date'].dt.to_period('M')
check_df['diff'] = check_df['diff'].apply(lambda x: x.n)

C:\Users\emanlangit.AEONCREDIT\AppData\Local\Temp\ipykernel_3584\120919408.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  check_df['diff'] = check_df['maturitydate'].dt.to_period('M') - check_df['repayment_start_date'].dt.to_period('M')
C:\Users\emanlangit.AEONCREDIT\AppData\Local\Temp\ipykernel_3584\120919408.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  check_df['diff'] = check_df['diff'].apply(lambda x: x.n)


In [23]:
check_df['check 2'] = (check_df['diff'] + 1) == check_df['number_of_installments']
check_df['check 2'].value_counts()

C:\Users\emanlangit.AEONCREDIT\AppData\Local\Temp\ipykernel_3584\2972022101.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  check_df['check 2'] = (check_df['diff'] + 1) == check_df['number_of_installments']


check 2
True    87093
Name: count, dtype: int64

In [27]:
#working_pdf.head()
reduced_pdf.head()
working_pdf = reduced_pdf.copy()

In [28]:
#merging dataset
merged_df = working_pdf.merge(check_df,how='inner',on=['loan_account_no'])

In [29]:
merged_df.columns

Index(['loan_account_no', 'report_date', 'number_of_installments_x',
       'days_past_due', 'bucket_MA', 'bucket_RBI', 'maturitydate',
       'number_of_installments_y', 'repayment_start_date', 'diff', 'check 2'],
      dtype='object')

In [30]:
reduced_merged_df = merged_df[['loan_account_no','repayment_start_date','report_date','bucket_MA','days_past_due']]

In [31]:
reduced_merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1139065 entries, 0 to 1139064
Data columns (total 5 columns):
 #   Column                Non-Null Count    Dtype         
---  ------                --------------    -----         
 0   loan_account_no       1139065 non-null  object        
 1   repayment_start_date  1139065 non-null  datetime64[ns]
 2   report_date           1139065 non-null  object        
 3   bucket_MA             1139065 non-null  object        
 4   days_past_due         1139065 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 43.5+ MB


In [32]:
reduced_merged_df['report_date'] = reduced_merged_df['report_date'].apply(lambda x: pd.to_datetime(x, format='%Y-%m-%d', errors = 'coerce'))

C:\Users\emanlangit.AEONCREDIT\AppData\Local\Temp\ipykernel_3584\2076558747.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_merged_df['report_date'] = reduced_merged_df['report_date'].apply(lambda x: pd.to_datetime(x, format='%Y-%m-%d', errors = 'coerce'))


In [33]:
rm_month_df = reduced_merged_df[['loan_account_no','days_past_due']]
rm_month_df['repayment_start_month'] = reduced_merged_df['repayment_start_date'].dt.to_period('M')
rm_month_df['report_month'] = reduced_merged_df['report_date'].dt.to_period('M')
rm_month_df['bucket_MA'] = reduced_merged_df['bucket_MA']
rm_month_df['nth month'] = rm_month_df['report_month'] - rm_month_df['repayment_start_month']
rm_month_df['nth month'] = rm_month_df['nth month'].apply(lambda x: x.n)
rm_month_df['nth month'] = rm_month_df['nth month'] + 1

C:\Users\emanlangit.AEONCREDIT\AppData\Local\Temp\ipykernel_3584\1376891725.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rm_month_df['repayment_start_month'] = reduced_merged_df['repayment_start_date'].dt.to_period('M')
C:\Users\emanlangit.AEONCREDIT\AppData\Local\Temp\ipykernel_3584\1376891725.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rm_month_df['report_month'] = reduced_merged_df['report_date'].dt.to_period('M')


In [34]:
rm_month_df

,loan_account_no,days_past_due,repayment_start_month,report_month,bucket_MA,nth month
0,TW01030102000002194455,0.0,2022-03,2022-01,D0,-1
1,UT01010104000002195645,0.0,2022-03,2022-01,D0,-1
2,TW01030107000002194434,0.0,2022-03,2022-01,D0,-1
3,TW01010103000002194591,0.0,2022-03,2022-01,D0,-1
4,TW01010104000002194367,0.0,2022-03,2022-01,D0,-1
...,...,...,...,...,...,...
1139060,TW01020103000002281621,0.0,2024-01,2024-03,D0,3
1139061,TW01010102000002212057,0.0,2022-09,2024-03,D0,19
1139062,TW01010107000002248785,0.0,2023-07,2024-03,D0,9
1139063,TW01010105000002284748,0.0,2024-02,2024-03,D0,2


In [35]:
rm_month_df.to_csv('working_df_v2.csv',index = False)

In [ ]:
rm_month_df

In [ ]:
ty = pd.read_csv(r"C:\Users\emanlangit\Documents\ACSIN\Notebooks\working_df_v2.csv")

In [ ]:
df = ty[ty['nth month'] > 0]

In [ ]:
df

In [ ]:
c1_6m =df[(df['loan_account_no'] == 'TW01030102000002194455') &  (df['nth month'] < 6)]
#c1_12m =df[(df['loan_account_no'] == 'TW01030102000002194455') &  (df['nth month'] < 12)] 

In [ ]:
Dscores_list_6 = c1_6m.bucket_MA.unique().tolist()

In [ ]:
c1_6m['month6_default'] = np.where(
    Dscores_list.str.contains("3"), 1, 0 
)

# c1_12m['month12_default'] = np.where(
#     c1_6m['bucket_MA'].str.contains("3"), 1, 0 
# )

In [ ]:
if 'D3' in Dscores_list:
    c1_6m['month6_default'] = 1
else:
    c1_6m['month6_default'] = 0
    
# if 'D3' in Dscores_list:
#     c1_6m['month6_default'] = 1
# else:
#     c1_6m['month6_default'] = 0

In [ ]:
c1_6m

In [ ]:
ser = c1_6m[['loan_account_no','month6_default']].drop_duplicates().iloc[0].tolist()

In [ ]:
tagged_customers = []

In [ ]:
tagged_customers.append(ser)

In [ ]:
tagged_customers

In [ ]:
ser

In [ ]:
#whole
customer_no_list = df.loan_account_no.unique().tolist()
tagged_customers_6m = []

for customer in customer_no_list:
    c1_6m =df[(df['loan_account_no'] == customer) &  (df['nth month'] <= 6)] #mini df per customer
    Dscores_list_6 = c1_6m.bucket_MA.unique().tolist() #get all Dscores of all instances of customer within period
    
    if 'D3' in Dscores_list:
        c1_6m['month6_default'] = 1
    else:
        c1_6m['month6_default'] = 0
    
    #get unique customer number and new target
    ser = c1_6m[['loan_account_no','month6_default']].drop_duplicates().iloc[0].tolist() 
    tagged_customers_6m.append(ser)
    
print('DONE')

In [ ]:
tagged_customers_6m

In [ ]:
tagged_6months_df = pd.DataFrame(tagged_customers_6m, 
                                 columns=['loan_customer_no', 'D3_within6months'])
tagged_6months_df.D3_within6months.value_counts()

In [ ]:
tagged_6months_df.D3_within6months.value_counts()

In [ ]:
df.loan_account_no

In [ ]:
#whole
#customer_no_list = df.loan_account_no[:30].unique().tolist()
tagged_customers_6m_sample = []

customer_no_list_sample = ['UT01010104000002195645', 'TW01010102000002194847', 'UT01010107000002273677']

for customer in customer_no_list_sample:
    c1_6m =df[(df['loan_account_no'] == customer) &  (df['nth month'] <= 6)] #mini df per customer
    Dscores_list_6 = set(c1_6m.bucket_MA.unique()) #get all Dscores of all instances of customer within period
    
    if 'D3' in Dscores_list_6 or 'D4' in Dscores_list_6:
        c1_6m['month6_default'] = 1
    else:
        c1_6m['month6_default'] = 0
    
    #get unique customer number and new target
    ser = c1_6m[['loan_account_no','month6_default']].drop_duplicates().iloc[0].tolist() 
    tagged_customers_6m_sample.append(ser)
    
    index = customer_no_list_sample.index(customer)
    print(index)
    
print('DONE')

In [ ]:
tagged_customers_6m_sample

In [ ]:
c1_6m =df[(df['loan_account_no'] == 'UT01010107000002273677') &  (df['nth month'] <= 6)] #mini df per customer
Dscores_list_6 = set(c1_6m.bucket_MA.unique()) #get all Dscores of all instances of customer within period
target = ['D3','D4 above']

if 'D3' in Dscores_list_6 or 'D4' in Dscores_list_6:
    c1_6m['month6_default'] = 1
else:
    c1_6m['month6_default'] = 0

In [ ]:
#whole_updated
customer_no_list = df.loan_account_no.unique().tolist()
tagged_customers_6m = []

for customer in customer_no_list:
    c1_6m =df[(df['loan_account_no'] == customer) &  (df['nth month'] <= 6)] #mini df per customer
    Dscores_list_6 = set(c1_6m.bucket_MA.unique()) #get all Dscores of all instances of customer within period
    
    if 'D3' in Dscores_list_6 or 'D4 above' in Dscores_list_6:
        c1_6m['month6_default'] = 1
    else:
        c1_6m['month6_default'] = 0
    
    #get unique customer number and new target
    ser = c1_6m[['loan_account_no','month6_default']].drop_duplicates().iloc[0].tolist() 
    tagged_customers_6m.append(ser)
    
    index = customer_no_list.index(customer)
    print(index)
    
print('DONE')

In [ ]:
tagged_6months_df = pd.DataFrame(tagged_customers_6m, 
                                 columns=['loan_customer_no', 'D3_within6months'])
tagged_6months_df.D3_within6months.value_counts()

In [ ]:
tagged_6months_df.to_csv('tagged_6months.csv',index = False)

In [ ]:
df[df['loan_account_no'] == 'TW01010103000002196469']

In [ ]:
basis_df['reachD3'].value_counts()

In [ ]:
df['bucket_days_past_due_based'] = np.where (df['days_past_due'] >= 91, 'D3', 'D0')

In [ ]:
#whole_updated
customer_no_list = df.loan_account_no.unique().tolist()
tagged_customers_6m = []

for customer in customer_no_list:
    c1_6m =df[(df['loan_account_no'] == customer) &  (df['nth month'] <= 6)] #mini df per customer
    Dscores_list_6 = set(c1_6m.bucket_days_past_due_based.unique()) #get all Dscores of all instances of customer within period
    
    if 'D3' in Dscores_list_6:
        c1_6m['month6_default'] = 1
    else:
        c1_6m['month6_default'] = 0
    
    #get unique customer number and new target
    ser = c1_6m[['loan_account_no','month6_default']].drop_duplicates().iloc[0].tolist() 
    tagged_customers_6m.append(ser)
    
    index = customer_no_list.index(customer)
    print(index)
    
print('DONE')